In [5]:
import os
from pathlib import Path

import cv2
import numpy as np
import mediapipe as mp

# 0. Quiet TensorFlow/absl
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 1. MediaPipe Pose setup
pose = mp.solutions.pose.Pose(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 2. Paths
VIDEO_ROOT = Path("Data-REHAB24-6/videos")
OUT_ROOT   = Path("Data-REHAB24-6/mp_keypoints")
OUT_ROOT.mkdir(exist_ok=True)

# 3. Worker
def process_video(vid_path: Path):
    rel     = vid_path.parent.name            # e.g. "Ex1"
    out_dir = OUT_ROOT / rel
    out_dir.mkdir(parents=True, exist_ok=True)
    out_file = out_dir / f"{vid_path.stem}-mp.npy"

    print(f"\n→ Processing: {vid_path.name}")
    print(f"   From:      {vid_path.parent}")
    print(f"   To folder: {out_dir}")

    cap    = cv2.VideoCapture(str(vid_path))
    frames = []
    count  = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        count += 1

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = pose.process(img)
        lm  = res.pose_world_landmarks.landmark if res.pose_world_landmarks else []

        if lm:
            pts = [(p.x, p.y, p.z) for p in lm]
        else:
            pts = [(0.0, 0.0, 0.0)] * 33

        frames.append(pts)

    cap.release()

    arr = np.array(frames, dtype=np.float32)
    np.save(out_file, arr)

    print(f"✔ Saved: {out_file}  (frames={count}, shape={arr.shape})")

# 4. Run — only Ex1 through Ex5
for i in range(1, 6):
    ex_dir = VIDEO_ROOT / f"Ex{i}"
    if not ex_dir.is_dir():
        print(f"⚠️  Skipping missing folder: {ex_dir}")
        continue

    for vid in sorted(ex_dir.glob("*.mp4")):
        try:
            process_video(vid)
        except Exception as e:
            print(f"✘ Failed processing {vid.name}: {e}")

print("\nAll requested videos processed.")


I0000 00:00:1744962682.601706  494303 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Max
W0000 00:00:1744962682.662757  558120 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744962682.684668  558120 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.



→ Processing: PM_000-Camera17-30fps.mp4
   From:      Data-REHAB24-6/videos/Ex1
   To folder: Data-REHAB24-6/mp_keypoints/Ex1
✔ Saved: Data-REHAB24-6/mp_keypoints/Ex1/PM_000-Camera17-30fps-mp.npy  (frames=1304, shape=(1304, 33, 3))

→ Processing: PM_001-Camera17-30fps.mp4
   From:      Data-REHAB24-6/videos/Ex1
   To folder: Data-REHAB24-6/mp_keypoints/Ex1
✔ Saved: Data-REHAB24-6/mp_keypoints/Ex1/PM_001-Camera17-30fps-mp.npy  (frames=3016, shape=(3016, 33, 3))

→ Processing: PM_002-Camera17-30fps.mp4
   From:      Data-REHAB24-6/videos/Ex1
   To folder: Data-REHAB24-6/mp_keypoints/Ex1
✔ Saved: Data-REHAB24-6/mp_keypoints/Ex1/PM_002-Camera17-30fps-mp.npy  (frames=1996, shape=(1996, 33, 3))

→ Processing: PM_012-Camera17-30fps.mp4
   From:      Data-REHAB24-6/videos/Ex1
   To folder: Data-REHAB24-6/mp_keypoints/Ex1
✔ Saved: Data-REHAB24-6/mp_keypoints/Ex1/PM_012-Camera17-30fps-mp.npy  (frames=2403, shape=(2403, 33, 3))

→ Processing: PM_016-Camera17-30fps.mp4
   From:      Data-REHAB24-